In [2]:
# setting the environment variables
import sys
import os

sys.path.insert(0, os.path.abspath('..'))

from config import set_environment

set_environment()

## LLMs

In [3]:
from langchain_openai import OpenAI

llm = OpenAI()

In [ ]:
summary = llm.invoke("Tell me a joke about light bulbs!")

In [ ]:
summary

## Chat Models

In [ ]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model_name='gpt-4-0613')
response = llm.invoke([HumanMessage(content='Say "Hello world" in Python.')])
print(response)

In [ ]:
type(response)

In [ ]:
from langchain_core.messages import SystemMessage

chat_output = llm.invoke([ 
    SystemMessage(content="You're a helpful assistant"), 
    HumanMessage(content="What is the purpose of model regularization?") 
])
print(chat_output)

In [ ]:
type(chat_output)

## Prompts

In [4]:
prompt = """
Summarize this text in one sentence:

{text}
"""

llm = OpenAI()
summary = llm(prompt.format(text="Some long story"))


/var/folders/0z/xsrm9ns529l62npj0_g400h80000gn/T/ipykernel_12440/1832409958.py:8: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = llm(prompt.format(text="Some long story"))


In [5]:
print(summary)


The text is about a long story.


In [6]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template("Tell me a {adjective} joke about {content}.") 


In [7]:
formatted_prompt = prompt_template.format(adjective="funny", content="chickens") 

In [8]:
print(formatted_prompt)

Tell me a funny joke about chickens.


In [9]:
prompt_val = prompt_template.invoke({"adjective": "funny", "content": "chickens"}) 

In [10]:
prompt_val

StringPromptValue(text='Tell me a funny joke about chickens.')

In [11]:
from langchain_openai.chat_models import ChatOpenAI 
from langchain.prompts import ChatPromptTemplate
# Define a ChatPromptTemplate to translate text
template = ChatPromptTemplate.from_messages([
    ('system', 'You are an English to French translator.'),
    ('user', 'Translate this to French: {text}')
])
llm = ChatOpenAI()
# Translate a joke about light bulbs
response = llm.invoke(template.format_messages(text='How many programmers does it take to change a light bulb?'))


In [ ]:
print(response)

## LangChain Expression Language

In [13]:
#from langchain_community.llms import HuggingFaceEndpoint 
from langchain.prompts import PromptTemplate 
from langchain.schema import StrOutputParser
#repo_id = "mistralai/Mistral-7B-Instruct-v0.2" 
llm = ChatOpenAI()
template = """Question: {question} Answer: Let's think step by step.""" 
prompt = PromptTemplate.from_template(template) 
runnable = prompt | llm | StrOutputParser()


In [14]:
question = "Who won the FIFA World Cup in the year 1994? " 
summary = runnable.invoke({"question": question})

In [15]:
summary

'The FIFA World Cup in 1994 was won by Brazil.'

## Text to Image

In [ ]:
from langchain.chains import LLMChain 
from langchain.prompts import PromptTemplate 
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper 
from langchain_openai import OpenAI

llm = OpenAI(temperature=0.9) 
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template=(
        "Generate a concise prompt to generate an image based on the following description: "
        "{image_desc}"
))
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
prompt = chain.run("halloween night at a haunted museum")
print(prompt)

In [ ]:
image_url = DallEAPIWrapper().run(chain.run("halloween night at a haunted museum"))

In [ ]:
image_url

In [ ]:
from IPython.display import Image, display
display(Image(url=image_url))


In [ ]:
import requests
response = requests.get(image_url)
image_path = "haunted_house.png"
with open(image_path, "wb") as f:
    f.write(response.content)


## Image to Text

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-4-turbo", max_tokens=256)
chat.invoke([
    HumanMessage(
        content=[
            {"type": "text", "text": "What is this image showing"}, 
            {
                "type": "image_url", 
                "image_url": { "url": image_url, "detail": "auto", },
            }, 
        ]
    )
])


In [ ]:
langchain_image = """
The image appears to be a diagram representing the architecture or components of a software system or framework related to language processing, possibly named LangChain or associated with a project or product called LangChain, based on the prominent appearance of that term. The diagram is organized into several layers or aspects, each containing various elements or modules:\n\n1. **Protocol**: This may be the foundational layer, which includes "LCEL" and terms like parallelization, fallbacks, tracing, batching, streaming, async, and composition. These seem related to communication and execution protocols for the system.\n\n2. **Integrations Components**: This layer includes "Model I/O" with elements such as the model, output parser, prompt, and example selector. It also has a "Retrieval" section with a document loader, retriever, embedding model, vector store, and text splitter. Lastly, there\'s an "Agent Tooling" section. These components likely deal with the interaction with external data, models, and tools.\n\n3. **Application**: The application layer features "LangChain" with chains, agents, agent executors, and common application logic. This suggests that the system uses a modular approach with chains and agents to process language tasks.\n\n4. **Deployment**: This contains "Lang'"
"""

In [ ]:
# prompt = chain.run("halloween night at a haunted museum")
# print(prompt)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template=(
        "Simplify this image description into a concise prompt to generate an image: "
        "{image_desc}"
))
chain = LLMChain(llm=llm, prompt=prompt)
prompt = chain.run(langchain_image)
print(prompt)

In [ ]:
image_url = DallEAPIWrapper().run(prompt)

In [ ]:
from IPython.display import Image, display
display(Image(url=image_url))